In [113]:
!pip install kagglehub tensorflow lime shap

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_in

In [114]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import kagglehub
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import lime
import shap
from lime.lime_tabular import LimeTabularExplainer

Dataset 1: Skin Cancer (Image Classification)

In [115]:
skin_cancer_path = kagglehub.dataset_download("johnsmith88/heart-disease-dataset")

Dataset 2: Heart Disease

In [ ]:
heart_disease_path = kagglehub.dataset_download('johnsmith88/heart-disease-dataset')
heart_disease_file = os.path.join(heart_disease_path, "heart.csv")
heart_data = pd.read_csv(heart_disease_file)

In [117]:
# Подготовка числового датасета
X_heart = heart_data.drop(columns=['target'])
y_heart = heart_data['target']

# Нормализация данных
scaler = StandardScaler()
X_heart_scaled = scaler.fit_transform(X_heart)

In [118]:
# Разделение на обучающую и тестовую выборки
X_train_heart, X_test_heart, y_train_heart, y_test_heart = train_test_split(
    X_heart_scaled, y_heart, test_size=0.2, random_state=42
)

In [119]:
# Построение модели для классификации болезней сердца
heart_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_heart.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

heart_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
heart_history = heart_model.fit(
    X_train_heart, y_train_heart, validation_split=0.2, epochs=50, batch_size=16, verbose=1
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


KeyboardInterrupt: 

In [ ]:
# Вывод точности для модели Heart Disease
heart_accuracy = heart_model.evaluate(X_test_heart, y_test_heart, verbose=0)[1]
print(f"Точность модели Heart Disease: {heart_accuracy:.2f}")

In [ ]:
# LIME для Heart Disease
lime_explainer = LimeTabularExplainer(
    training_data=X_train_heart,
    feature_names=heart_data.drop(columns=['target']).columns,
    class_names=['No Disease', 'Disease'],
    mode='classification'
)

lime_exp = lime_explainer.explain_instance(
    X_test_heart[2],
    heart_model.predict,
    num_features=13,
    labels=(0,)
)
lime_exp.show_in_notebook()

In [ ]:
shap_explainer = shap.KernelExplainer(heart_model.predict, X_train_heart)
shap_values = shap_explainer.shap_values(X_test_heart[:2])

In [ ]:
shap.summary_plot(shap_values.reshape(2, 13), X_test_heart[:2], feature_names=heart_data.drop(columns=['target']).columns)

In [ ]:
skin_cancer_path = kagglehub.dataset_download("nodoubttome/skin-cancer9-classesisic")

data_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_gen = data_gen.flow_from_directory(
    skin_cancer_path + '/Skin cancer ISIC The International Skin Imaging Collaboration/Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = data_gen.flow_from_directory(
    skin_cancer_path  + '/Skin cancer ISIC The International Skin Imaging Collaboration/Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Заморозим веса предобученной части

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_gen.num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    verbose=1
)


In [ ]:
test_data_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Предобработка для ResNet50
    validation_split=0.99  # 20% данных для валидации
)

test_gen = test_data_gen.flow_from_directory(
    skin_cancer_path  + '/Skin cancer ISIC The International Skin Imaging Collaboration/Test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Оценка модели на валидационных данных
val_loss, val_accuracy = model.evaluate(test_gen, verbose=0)
print(f"Точность на валидационной выборке: {val_accuracy:.2f}")
